In [1]:
from pandas import DataFrame, Series
import pandas as pd; import numpy as np

In [5]:
#Get data only for users who have posted reviews in categories Chinese^(restaurant v food)
import json
USERPATH='../full_dataset/user.json'
FS1PATH='../full_dataset/featureset1_resto_food_chinese.json'
userframe = DataFrame([json.loads(line) for line in open(USERPATH)])[['user_id', 'friends']]
active_users_frame = pd.read_json(FS1PATH)
userframe = pd.merge(userframe, active_users_frame, on='user_id')[['user_id', 'friends']]
active_users_frame = None
userframe.head() #Keep users w/out friends: they may still post reviews, which we want to count.

,categories_avg_rating,categories_biz_count,categories_reviews,std_dev_rating,total_reviews,user_id
0,5,1,1,0,4,-BjekrEUhweLVyrjVStMgg
1,5,1,1,0,6,XQiaNhoZbdxaN3pJIBADvw
10,2,1,1,0,12,82d-mm3hB2UelHwrptvcMw
100,2,1,1,0,22,0XMC32Lkh6x1Yc9m9sSK0Q
1000,1,1,1,0,1,_j907SJO1Tzl102sXiJPLA


In [3]:
#Flatten "friends" field, merge back into user-friend matrix
flatteningframe = DataFrame([[i, friend]
                                     for i, friends in userframe['friends'].apply(list).iteritems()
                                         for friend in friends],
                                columns=['index', 'friend_id'])
flatteningframe = flatteningframe.set_index('index')
graph_base_frame = pd.merge(userframe, flatteningframe, left_index = True, right_index = True)[['user_id', 'friend_id']]
graph_base_frame.head()

,user_id,friend_id
0,UTS9XcT14H2ZscRIf0MYHQ,rpOyqD_893cqmDAtJLbdog
0,UTS9XcT14H2ZscRIf0MYHQ,4U9kSBLuBDU391x6bxU-YA
0,UTS9XcT14H2ZscRIf0MYHQ,Re447krbp0VQVX3Dk4Tsog
0,UTS9XcT14H2ZscRIf0MYHQ,kJc9YBRwmmZ_PG0uLHuEPQ
0,UTS9XcT14H2ZscRIf0MYHQ,Rir-YRPPClKXDFQbc3BsVw


In [4]:
flatteningframe = None
len(graph_base_frame.index)

845530

In [ ]:
#TODO: Select only authorities for this step
user_user = pd.merge(graph_base_frame, graph_base_frame, left_on='user_id', right_id='friend_id')
